In [63]:
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from konlpy.tag import Twitter
from bs4 import BeautifulSoup
import requests
import tensorflow as tf
import re
import glove
from sklearn.feature_extraction.text import CountVectorizer

### 매우 간단한 word2vec 구현!! 

In [64]:
corpus = [['HE', 'IS', 'A', 'KING'],
         ['SHE', 'IS', 'A', 'QUEEN']]

In [65]:
words = ['HE', 'IS', 'A', 'KING', 'SHE', 'QUEEN']

In [66]:
HE = np.array([1,0,0,0,0,0])
IS = np.array([0,1,0,0,0,0])
A = np.array([0,0,1,0,0,0])
KING = np.array([0,0,0,1,0,0])
SHE = np.array([0,0,0,0,1,0])
QUEEN = np.array([0,0,0,0,0,1])

In [92]:
# window size=2로 트레인셋 구성 
data = np.array( 
    # 첫번째 말뭉치
    [[HE, IS],
    [HE, A],
    [IS, HE],
    [IS, A],
    [IS, KING],
    [A, HE],
    [A, IS],
    [A, KING],
    [KING, IS],
    [KING, A],
     # 두번째 말뭉치 ! 
    [SHE, IS],
    [SHE, A],
    [IS, SHE],
    [IS, A],
    [IS, QUEEN],
    [A, SHE],
    [A, IS],
    [A, QUEEN],
    [QUEEN, IS],
    [QUEEN, A],])

In [93]:
x =data[0][0]
y =data[0][1]
for datum in data[1:]:
    x = np.vstack([x, datum[0]])
    y = np.vstack([y, datum[1]])

In [94]:
x

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [95]:
y

array([[0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [96]:
n_words=6
size=20

train_X = tf.placeholder(shape=[None, n_words], dtype=tf.float32)
Y = tf.placeholder(shape=[None, n_words], dtype=tf.float32)

In [97]:
w1 = tf.Variable(tf.random_normal([n_words, size], dtype=tf.float32))
b1 = tf.Variable(tf.random_normal([size], dtype=tf.float32))
hidden = tf.add(tf.matmul(train_X, w1), b1)
w2 = tf.Variable(tf.random_normal([size, n_words], dtype=tf.float32))
b2 = tf.Variable(tf.random_normal([n_words], dtype=tf.float32))
output = tf.add(tf.matmul(hidden, w2), b2)

In [98]:
pred = tf.nn.softmax(output)
cost = tf.reduce_mean(tf.reduce_sum(-tf.log(pred)*y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
training = optimizer.minimize(cost)

In [99]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
sess.run(w1)

array([[ 0.858817  ,  0.29076764, -0.33557764, -0.27250582, -0.21607119,
        -0.3547673 ,  1.4801834 ,  0.5703814 , -1.5143    , -1.0327244 ,
        -1.400462  , -0.08717239, -0.36310494,  1.805142  , -0.17638384,
         0.00822871,  1.1657928 ,  0.11531438, -0.09092948,  0.29194418],
       [-2.1451807 , -1.7408028 , -0.90330213,  0.2938923 , -0.43620375,
        -0.07149015, -0.70787805, -1.6201991 ,  0.67210895,  0.3454133 ,
         0.7134726 , -0.75154364,  1.066471  ,  0.32260513,  0.2624124 ,
         1.8396635 , -1.0893948 , -1.487749  ,  1.9604955 , -0.3670063 ],
       [ 2.1941733 ,  0.48256353, -0.8552824 , -0.19386373, -0.6446642 ,
        -0.3415236 ,  1.0965323 ,  1.99009   ,  0.6132252 , -0.7805163 ,
         0.36202997,  0.01579673,  0.8057624 ,  0.2524156 ,  2.0983596 ,
        -2.2505617 , -1.3164946 , -1.2302628 ,  0.25833434,  1.4723723 ],
       [ 0.01967947,  0.3216652 ,  0.1308611 ,  0.07121683,  0.12359463,
        -0.05699532, -1.1494192 , -0.2671397 ,  

In [100]:
for _ in range(1000):
    sess.run(training, feed_dict={train_X:x, Y:y})
    if _%100 ==0 :
        print('loss is', sess.run(cost, feed_dict={train_X:x, Y:y}))

loss is 119.24999
loss is 34.42814
loss is 26.679352
loss is 24.921055
loss is 24.659817
loss is 24.552414
loss is 24.493412
loss is 24.455917
loss is 24.42985
loss is 24.410587


In [101]:
idx = sess.run(w1 + b1)
idx

array([[ 0.36249575,  1.7865481 ,  0.05759183, -1.4081398 , -0.08682993,
        -2.1308103 ,  1.3548332 ,  0.41068113, -2.385829  , -0.6249325 ,
        -1.6170515 ,  0.8274981 , -0.14059433,  2.5505643 ,  0.23844421,
         0.800745  ,  0.96323013,  0.40107378, -0.88455373, -2.3765588 ],
       [-2.008899  , -0.33457816, -0.5557617 , -0.754636  , -0.75413555,
        -1.4069964 , -0.48537233, -2.1617005 ,  0.42314178,  0.74718916,
         1.2055098 , -0.20656574,  0.83159983,  1.8836292 ,  0.11195521,
         2.4897513 , -1.0276372 , -1.113575  ,  0.9069782 , -2.934667  ],
       [ 2.0996969 ,  1.9776193 , -0.40776512, -1.076443  , -0.82632875,
        -1.9072455 ,  1.284887  ,  1.1295447 ,  0.07703543, -0.3013233 ,
         0.59627014,  0.7470697 ,  0.32474804,  1.6139226 ,  1.9708999 ,
        -1.5368578 , -1.2120781 , -1.0710698 , -0.80511594, -1.1607012 ],
       [-0.4606896 ,  1.9614198 ,  0.02441695, -1.2569313 ,  0.47665077,
        -1.7651075 , -1.2992462 , -0.1320968 ,  

In [102]:
words

['HE', 'IS', 'A', 'KING', 'SHE', 'QUEEN']

In [103]:
my_w2v = dict(zip(words,idx))
my_w2v

{'A': array([ 2.0996969 ,  1.9776193 , -0.40776512, -1.076443  , -0.82632875,
        -1.9072455 ,  1.284887  ,  1.1295447 ,  0.07703543, -0.3013233 ,
         0.59627014,  0.7470697 ,  0.32474804,  1.6139226 ,  1.9708999 ,
        -1.5368578 , -1.2120781 , -1.0710698 , -0.80511594, -1.1607012 ],
       dtype=float32),
 'HE': array([ 0.36249575,  1.7865481 ,  0.05759183, -1.4081398 , -0.08682993,
        -2.1308103 ,  1.3548332 ,  0.41068113, -2.385829  , -0.6249325 ,
        -1.6170515 ,  0.8274981 , -0.14059433,  2.5505643 ,  0.23844421,
         0.800745  ,  0.96323013,  0.40107378, -0.88455373, -2.3765588 ],
       dtype=float32),
 'IS': array([-2.008899  , -0.33457816, -0.5557617 , -0.754636  , -0.75413555,
        -1.4069964 , -0.48537233, -2.1617005 ,  0.42314178,  0.74718916,
         1.2055098 , -0.20656574,  0.83159983,  1.8836292 ,  0.11195521,
         2.4897513 , -1.0276372 , -1.113575  ,  0.9069782 , -2.934667  ],
       dtype=float32),
 'KING': array([-0.4606896 ,  1.961

### w2v 실습

In [79]:
# 크롤링 
reviews=[]
for i in range(1,100):
    r = requests.get('https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=172454&target=after&page=' + str(i))
    soup = BeautifulSoup(r.content, 'html.parser')
    items = soup.findAll("td", {"class" : "title"})
    for item in items:
        review = item.text.replace("\n", "").replace("\r","").replace("\t","").replace("신고","")
        reviews += [review]

In [21]:
reviews[:1000]

['곤지암한국 공포영화중 최고, 스토리, 몰입감, 잔잔히 젖어드는 무서움(아직도 생각나네요), 연기력, 연출 하지만 아쉬웠던 엔딩! ',
 '곤지암쓸데없는 사연이나 진부한 설정으로 질질 빼지 않아 지루하지 않음. 딱 기분 좋게 공포만을 즐길 수 있는 영화 ',
 '곤지암공포영화 무서워하는분들에겐 볼만할텐데 잘보는 사람들에겐 좀 지루할수도.. 초반엔 기대하지 마시고 마지막에 한꺼번에 재밌는거 몰아침 여튼 저는 그냥 그럭저럭 이었습니다.. 뭐 평소에도 공포영화 잘보긴했지만 ㅋㅋㅋ.. 근데 진짜 학생들 너무 시끄러워요 ',
 '곤지암최근에 본 영화중에 무섭고, 재밌었어요. ',
 '곤지암그냥 너무 무섭다는 말 말곤.. 너무 무서워요. 영화보다 나가고 싶다라는 생각 첨해봤어여 ',
 '곤지암중간중간 뒷사람 피식거리고 영화내내 한번도 안놀란거면 1점도 과분하다. 내돈주고 봤지만 같이 봐준 남자친구한테 미안할정도. ',
 '곤지암솔직히... 처음엔 긴장감돌고 깜짝놀라기도했는데... 끝은왜이리허무하던지... 공포영화는 엑소시스트가갑이였다... ',
 '곤지암영화보고난후 소감 : 아연이 예쁘다 끝 ',
 '곤지암뒤로가면 진짜 무서움 ㄷㄷ ',
 '곤지암호롤롤롤롤로!!!! ',
 '곤지암후반부 지림..시붑시시스부한다는거 여기댓글로 보고갔는데. 나오자마자 소름돋음ㄷㄷ공포영화좋아하시는분들에게 강추어떤 남성분 영화끝나고 친구들한테 팬티대신 겨드랑이 보여주심. 다 젖었다고.. ',
 '곤지암쓰레기영화 최악이네요 ',
 '곤지암새로운 시도는 좋았으나 무섭진않다. ',
 '곤지암어디서 놀라야하나 ㅠ. ',
 '곤지암여자친구와 봣는데 여자친구는 무서워 하고 저는 재미없게 봣어요 ',
 '곤지암초반에는 하품이 절로 나는데 후반엔 좀 달리다가 뚝 끊기는 느낌?연출력이나 배우들의 참신한 연기력을 감안하면 9점 정도 주고 싶지만 돈주고 본 영화라는걸 감안하면 7점이 딱 적당한듯.클로버 필드와 유사한 시점이어서 식사 직후에 보면 쏠릴지도... ',
 '곤지암한국공포영화중 최고..무서워요. ',
 

In [22]:
twitter = Twitter()

In [23]:
reviews_words=[]
cnt ={}
for review in reviews:
    t=[]
    for word in twitter.pos(review[3:]):
        if word[1] in ["Noun", "Adjective"] and len(word[0])>=2:
            if word[0] not in ['진짜', '너무', '그냥', '정도', '보고', '정말', '입니', '제일']:
                t += [word[0]]
            if word[0] in cnt:
                cnt[word[0]] +=1
            else:
                cnt[word[0]]=1
    reviews_words += [t]
reviews_words[:1000]

[['한국', '공포영화', '최고', '스토리', '몰입', '무서움', '아직', '연기력', '연출', '아쉬웠', '엔딩'],
 ['쓸데없', '사연', '진부', '설정', '질질', '지루하지', '기분', '공포', '있는', '영화'],
 ['공포영화',
  '무서워',
  '사람',
  '지루할',
  '초반',
  '기대하지',
  '마지막',
  '재밌는',
  '그럭저럭',
  '평소',
  '공포영화',
  '학생',
  '시끄러워'],
 ['최근', '영화', '무섭', '재밌었'],
 ['무섭', '무서워', '영화', '생각'],
 ['중간', '중간', '사람', '거리', '영화', '내내', '한번', '점도', '과분', '봐준', '남자친구', '미안할'],
 ['솔직히', '처음', '긴장감', '깜짝', '허무', '공포영화', '엑소시스트'],
 ['영화', '난후', '소감', '아연', '예쁘'],
 ['가면', '무서움'],
 ['롤롤', '롤롤'],
 ['후반',
  '시붑시',
  '시스',
  '여기',
  '댓글',
  '소름',
  '공포영화',
  '좋아하시',
  '강추',
  '어떤',
  '남성',
  '영화',
  '친구',
  '팬티',
  '대신',
  '겨드랑이',
  '주심'],
 ['쓰레기', '영화', '최악'],
 ['새로운', '시도', '무섭'],
 [],
 ['여자친구', '봣는데', '여자친구', '무서워', '재미없', '봣어'],
 ['초반',
  '하품',
  '절로',
  '후반',
  '느낌',
  '연출',
  '배우',
  '참신',
  '연기력',
  '감안',
  '영화',
  '는걸',
  '감안',
  '적당한',
  '클로버',
  '필드',
  '유사한',
  '시점',
  '식사',
  '직후',
  '쏠릴지'],
 ['한국', '공포영화', '최고', '무서워'],
 ['지현', '시비', '비지', '싯시', '무서움', '결말', '아쉽'],
 

In [24]:
sorted(cnt.items(), key=lambda x : x[1], reverse=True)[:20]

[('영화', 324),
 ('무섭', 186),
 ('공포영화', 144),
 ('진짜', 141),
 ('너무', 125),
 ('사람', 104),
 ('공포', 92),
 ('귀신', 77),
 ('무서워', 73),
 ('무서운', 70),
 ('보고', 64),
 ('소리', 61),
 ('무서웠', 56),
 ('한국', 54),
 ('그냥', 51),
 ('노잼', 47),
 ('무서움', 46),
 ('정도', 44),
 ('정말', 44),
 ('배우', 43)]

In [32]:
model = Word2Vec(reviews_words, window=10, size = 10, min_count=20, sg=1)

In [104]:
Word2Vec?

In [34]:
model.wv.index2word

['영화',
 '무섭',
 '공포영화',
 '사람',
 '공포',
 '귀신',
 '무서워',
 '무서운',
 '소리',
 '무서웠',
 '한국',
 '노잼',
 '무서움',
 '배우',
 '평점',
 '중간',
 '곤지암',
 '재미없',
 '느낌',
 '처음',
 '연기',
 '스토리',
 '팝콘',
 '없고',
 '재미',
 '아깝',
 '생각',
 '아니',
 '별로',
 '장면',
 '마지막',
 '혼자',
 '샬롯',
 '최고',
 '있는',
 '알바',
 '내용',
 '영화관',
 '연출',
 '진심',
 '소름',
 '부분',
 '시브',
 '하나',
 '시간',
 '후반',
 '같은',
 '초반',
 '깜짝',
 '괜찮',
 '보지']

In [56]:
model.most_similar("영화관관")

KeyError: "word '영화관관' not in vocabulary"

In [36]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [37]:
w2v

{'같은': array([ 0.04795076, -0.00313362,  0.00808393, -0.02274017, -0.002771  ,
         0.05058543, -0.02617554,  0.03460472,  0.04717677,  0.00620821],
       dtype=float32),
 '곤지암': array([ 0.0062903 , -0.02265162, -0.0401005 ,  0.03148952,  0.0338664 ,
         0.00192896, -0.00277362,  0.00533999,  0.05703996,  0.04214407],
       dtype=float32),
 '공포': array([ 0.03950201,  0.02445409,  0.04078279,  0.0434348 , -0.0329515 ,
         0.0012838 ,  0.03198537, -0.00835164,  0.03037058,  0.01541869],
       dtype=float32),
 '공포영화': array([ 0.01680901, -0.05302242, -0.01221459, -0.0391608 ,  0.00175132,
         0.05914764,  0.02390466,  0.05341759,  0.05438143, -0.03938267],
       dtype=float32),
 '괜찮': array([-0.00028726, -0.03052084,  0.00421524,  0.04554198, -0.05975791,
         0.01448402,  0.06209891, -0.03076857, -0.02457345,  0.05118851],
       dtype=float32),
 '귀신': array([ 0.05867069,  0.02589288,  0.05645598, -0.04664647, -0.05105268,
         0.01338345, -0.03306034,  0.0

### Fast text 실습

In [38]:
model2 = FastText(reviews_words, max_n=6, min_n=3, size=10, sg=1, min_count=20)

In [39]:
FastText?

In [57]:
model2.most_similar("영화관관") # 존재하지 않는 단어도 계산 가능 

[('영화관', 0.851671576499939),
 ('재미없', 0.8331696391105652),
 ('후반', 0.8174705505371094),
 ('같은', 0.7390350103378296),
 ('아니', 0.7343484163284302),
 ('재미', 0.7209592461585999),
 ('배우', 0.7182731032371521),
 ('영화', 0.7166975736618042),
 ('시브', 0.7037427425384521),
 ('무섭', 0.6364915370941162)]

In [60]:
w2v_2 = dict(zip(model.wv.index2word, model.wv.syn0))

### glove 실습

In [50]:
# co-occurrence matrix 만들기
vectorizer = CountVectorizer(min_df=10, ngram_range=(1,1))
X = vectorizer.fit_transform(reviews)
Xc = X.T * X    

In [41]:
# co-occurrence matrix 만들기
vectorizer = CountVectorizer(min_df=10, ngram_range=(1,1))
X = vectorizer.fit_transform(reviews)
Xc = X.T * X             # co-occurrence matrix
Xc.setdiag(0)            # 대각성분을 0으로
result = Xc.toarray()    # array로 변환
dic = {}
for idx1, word1 in enumerate(result):
    tmpdic = {}
    for idx2, word2 in enumerate(word1):
        if word2 > 0:
            tmpdic[idx2] = word2
    dic[idx1] = tmpdic

In [49]:
dic

{0: {1: 4,
  5: 1,
  9: 1,
  12: 1,
  13: 1,
  15: 1,
  16: 1,
  18: 1,
  24: 1,
  30: 1,
  33: 1,
  36: 2,
  46: 1,
  47: 1,
  50: 2,
  51: 1,
  53: 1,
  54: 3,
  57: 2,
  59: 2,
  60: 1,
  61: 1,
  64: 1,
  65: 2},
 1: {0: 4,
  3: 1,
  7: 1,
  13: 1,
  16: 1,
  18: 1,
  19: 1,
  22: 1,
  31: 1,
  36: 1,
  47: 2,
  51: 2,
  53: 1,
  57: 1,
  59: 5,
  61: 2},
 2: {8: 1,
  22: 1,
  29: 1,
  30: 1,
  31: 1,
  35: 1,
  36: 1,
  50: 1,
  54: 1,
  55: 1,
  59: 3,
  65: 1},
 3: {1: 1,
  7: 1,
  11: 4,
  22: 1,
  35: 1,
  37: 1,
  38: 2,
  48: 1,
  51: 4,
  58: 1,
  59: 3},
 4: {6: 1,
  9: 1,
  14: 2,
  15: 2,
  16: 2,
  17: 2,
  19: 1,
  22: 5,
  24: 2,
  28: 1,
  35: 1,
  37: 1,
  38: 1,
  47: 1,
  48: 1,
  49: 1,
  50: 1,
  54: 2,
  59: 2,
  60: 1,
  66: 1},
 5: {0: 1,
  11: 1,
  12: 1,
  14: 1,
  15: 1,
  17: 1,
  19: 1,
  22: 2,
  26: 1,
  27: 1,
  30: 1,
  32: 1,
  36: 1,
  38: 1,
  44: 1,
  45: 1,
  49: 1,
  58: 1,
  59: 2,
  62: 1,
  63: 1,
  66: 1},
 6: {4: 1,
  12: 1,
  13: 1,
  17:

In [42]:
# 단어 리스트 작성
import operator
vocab = sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(1))
vocab = [word[0] for word in vocab]

In [43]:
model3 = glove.Glove(dic)

In [44]:
for epoch in range(25):
    err = model3.train(batch_size=200, workers=4)
    print("epoch %d, error %.3f" % (epoch, err), flush=True)

epoch 0, error 0.018
epoch 1, error 0.014
epoch 2, error 0.011
epoch 3, error 0.009
epoch 4, error 0.008
epoch 5, error 0.007
epoch 6, error 0.007
epoch 7, error 0.006
epoch 8, error 0.006
epoch 9, error 0.006
epoch 10, error 0.006
epoch 11, error 0.006
epoch 12, error 0.006
epoch 13, error 0.006
epoch 14, error 0.006
epoch 15, error 0.006
epoch 16, error 0.006
epoch 17, error 0.006
epoch 18, error 0.006
epoch 19, error 0.006
epoch 20, error 0.006
epoch 21, error 0.006
epoch 22, error 0.006
epoch 23, error 0.005
epoch 24, error 0.005


In [45]:
dict(zip(vocab, model3.W))

{'ㅋㅋ': array([-6.41815126e-03,  3.47812924e-03, -1.32501897e-03,  7.06336339e-03,
         7.46291990e-03, -5.16267588e-03, -5.01038162e-03,  5.61110874e-03,
         7.39063167e-03,  9.36977934e-03, -2.09951891e-03, -5.16100393e-04,
         4.42851240e-03,  2.93909171e-03, -9.95324993e-04,  5.74155427e-04,
        -1.33577514e-03, -9.22420213e-03,  5.37159292e-03,  6.89798346e-03,
        -3.40212928e-03,  2.72111806e-03, -9.74404545e-03, -3.18761292e-03,
         9.96525220e-03,  2.91615127e-03, -3.86765567e-04,  6.95366642e-03,
        -2.08852648e-03,  9.98967694e-04,  4.64101997e-03,  8.10764051e-04,
         6.31563013e-03, -7.87073898e-03,  4.93580183e-03,  3.64351702e-03,
        -5.82659867e-03,  8.48520330e-03, -1.29513082e-05,  9.94036331e-03,
        -7.95596498e-03, -5.38969645e-03, -8.54271318e-03,  5.02690218e-03,
         7.21627650e-04,  8.78984759e-04, -8.59402827e-03,  8.90784859e-04,
        -3.64287402e-03,  3.43782499e-04]),
 'ㅋㅋㅋ': array([-0.00804494,  0.0034534

In [46]:
wordvectors = model3.W

In [47]:
# 유사도 계산 함수
from scipy.spatial.distance import cosine
def most_similar(word, vocab, vecs, topn=10):
	query = vecs[vocab.index(word)]
	result = []
	for idx, vec in enumerate(vecs):
		if idx is not vocab.index(word):
			result.append((vocab[idx],1-cosine(query,vec)))
	result = sorted(result,key=lambda x: x[1],reverse=True)
	return result[:topn]

In [48]:
most_similar(word='영화', vocab=vocab, 
             vecs=wordvectors, topn=5)

[('ㅡㅡ', 0.34473860725105765),
 ('제일', 0.2347244068943779),
 ('아깝다', 0.2296519423684127),
 ('ㅠㅠ', 0.20854651974814264),
 ('하고', 0.18577419135737483)]